# Final Project Part III: Stream processing and full APRS tranciever


In this part of the project we will take the functions that we wrote in the previous part and put them in a streaming architecture such that we can collect finite buffers and process them in real-time. 

We found some issues with the current implementation of ax25.py. The changes we made break Part II. So in this part we will use the file newax25.py instead.

In [ ]:
# You can copy the compiled libpll.c from Part II or recompile pll.c with this command
!cc -fPIC -shared -o libpll.so pll.c

In [ ]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import threading
import time
import sys
import multiprocessing

from numpy import ones, zeros, pi, sin, cos, exp, sign, r_, mean, power
from numpy.fft import fft, fftshift, ifft, ifftshift
from scipy import signal
from scipy import integrate

import bitarray
from scipy.io.wavfile import read as wavread
import newax25 as ax25
from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
import base64

%matplotlib inline

In [ ]:
# function to compute least common multiple
def lcm(numbers):
    return reduce(lambda x, y: (x*y)//gcd(x,y), numbers, 1)

#### Pre-task 0: Audio Devices
Setup the software loopback and ensure that the various device indexes are correct, as in Part II.

In [ ]:
!sudo modprobe snd_aloop
!aplay -l | grep -i loopback

In [ ]:
sd.query_devices()

In [ ]:
builtin_idx = 0
usb_idx = 1
loop_idx = 3  # Use loopback dev 1! direwolf is connected to dev 0.
loop_idx_rcv = 2 # Use loopback dev 0. 
sd.default.samplerate = 48000
sd.default.channels = 1

#### Pre-task 1: Direwolf
Copy the direwolf config from Part II to this directory and start direwolf with ``./run-direwolf-loopback.sh`` in a separate terminal. Note that this bash script has a different name than in Part II; this is to differentiate it from a different script that you will run in Task 10.

## Creating a TNC (Terminal Node Controller, i.e. modem) class

The architecture we chose for this lab is to create a class that implements the functions of the modem, while keeping joint state variables such that it would be possible to process buffers in real-time while maintaining continuity and integrity of the processing as if we are processing a large buffer. 


We created the infrastructure for you, implemented in the `TNCaprs` class. 
It implements an overlap and save approach. The constructor for the class takes in the sampling frequency: `fs`, the size of soundDevice buffer (default is 1024): `Abuffer`, and the number of audio buffers to collect before processing: `Nchunks`.


##### `processBuffer(self, buff_in)`
The method `processBuffer(buff_in)` takes buffers in to be processed, collects them into a large buffer made of `Nchunks`. Once the large buffer has been collected, the method calls the demodulation method to create an NRZI, it calls the PLL to find the sampling timings, it samples the NRZI and converts it to NRZ bits. Finally it calls the function that looks for packets in the bitstream. 
The method implements an overlap and save approach. So, it calls the demodulation function with a buffer that is overlapping and expects a smaller buffer containing only valid linear convolution samples in return. 

##### `demod(self, buff)` -- same as `nc_afsk1200Demod`
Because there are 3 filters one after the other in the demodulator, each one of length N, the method `processBuffer(buff_in)` sends to `demod` a buffer sized `Nchunks*Abuffer + 3*(N-1)` and expects `Nchunks*Abuffer` samples in return. If you have different size filters, you need to modify `processBuffer(buff_in)` to account for that. The filters for `demod` are generated in the `__init__` function of the `TNCaprs` class. When filtering, use `np.convolve` with `mode='valid'` so that only valid linear convolution results are returned. 

##### ` PLL(self, NRZa)`
This is the same PLL implementation as before. The only difference is that the PLL counter: `pll`, its previous value: `ppll`, the stepsize: `dpll`, and the aggressiveness or nudge factor: `apll` are all class variables. This way, when the PLL finishes processing a buffer, the values are saved and used in the beginning of the next buffer.

##### ` FastPLL(self, NRZa)`
The same implementation of the PLL -- but a wrapper to a C-language function. This is much faster than the one written in python.

##### `findPackets(self,bits)`
This function is the same as before. The only differences are that the variables, `state`, `pktcounter`, and `packet`, are class variable and their value is kept from one call to another. 

Another difference is that we also added another variable: `bitpointer`. As you recall, the function looks for flags in the bitstream up to 7 bits before the end of the buffer. `bitpointer` is needed for the case where a flag is detected at the end of the buffer and extend to these 7 bits. That means that when we process the next buffer, we need to start after those bits, and `bitpointer` points to the bit we need to start with.

##### `modulate(self, bits)` -- same as `afsk1200`
Function takes bits and afsk1200 modulates them. The sampling rate is initialized in  `TNCaprs.__init__`.

##### `modulatePacket(self, callsign, digi, dest, info, preflags=80, postflags=80 )`
Given callsign, digipath, dest, info, number of pre-flags and post-flags, the function contructs an appropriate APRS packet, then converts it to NRZI and calls `modulate` to afsk1200 modulate the packet. 

#### Task 1: 
* Convert your functions `nc_afsk1200Demod` and  `afsk1200` to `TNCaprs.demod` and `TNCaprs.modulate`. Make it into class form in which class variables are in the form of `self.variable` which are initialized in `TNCaprs.__init__`. 

In [ ]:
import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

array_1d_int = npct.ndpointer(dtype=np.int, ndim=1, flags='CONTIGUOUS')

libcd = npct.load_library("./libpll", ".")
libcd.pll.restype = c_int
libcd.pll.argtypes= [array_1d_int, c_int, array_1d_int,array_1d_int,  array_1d_int,array_1d_int, c_int, c_float]


class TNCaprs: 
    def __init__(self, fs=48000.0, Abuffer=1024, Nchunks=10):
        
        #  Implementation of an afsk1200 TNC. 
        #
        #  The TNC processes `Abuffer` long buffers until `Nchunks` number of buffers are 
        #  collected into a large one.
        #  This is because python is able to more efficiently process larger buffers than smaller ones.
        #  Then, the resulting large buffer is demodulated, sampled, and packets extracted.
        #
        # Inputs:
        #   fs      - sampling rate
        #   TBW     - TBW of the demodulator filters
        #   Abuffer - Input audio buffers from Pyaudio
        #   Nchunks - Number of audio buffers to collect before processing
        #   apll    - nudge factor of the PLL
        
        ## compute sizes based on inputs
        self.TBW = 2.0   # TBW for the demod filters
        self.N = (int(fs/1200*self.TBW)//2)*2+1   # length of the mark-space filters for demod
        self.fs = fs     # sampling rate   
        self.BW = 1200      # BW of filter based on TBW
        self.Abuffer = Abuffer             # size of audio buffer
        self.Nchunks = Nchunks             # number of audio buffers to collect
        self.Nbuffer = Abuffer*Nchunks+(self.N*3-3)         # length of the large buffer for processing
        self.Ns = 1.0*fs/1200.0 # samples per symbol
        
        ## state variables for the modulator
        self.prev_ph = 0  # previous phase to maintain continuous phase when recalling the function
        
        ## generate filters for the demodulator
        self.h_lp = signal.firwin(self.N,self.BW/fs*1.0,window='hanning')
        self.h_lpp = signal.firwin(self.N,self.BW*2*1.2/fs,window='hanning')
        self.h_space = self.h_lp*exp(1j*2*pi*(2200)*(r_[0:self.N]-self.N//2)/fs)
        self.h_mark = self.h_lp*exp(1j*2*pi*(1200)*(r_[0:self.N]-self.N//2)/fs)
        self.h_bp = signal.firwin(self.N,self.BW/fs*2.2,window='hanning')*exp(1j*2*pi*1700*(r_[0:self.N]-self.N//2)/fs)

        ## PLL state variables  -- so continuity between buffers is preserved
        self.dpll = np.round(2.0**32 / self.Ns).astype(np.int32)    # PLL step
        self.pll = 0                # PLL counter
        self.ppll = -self.dpll       # PLL counter previous value -- to detect overflow
        self.apll = 0.74             # PLL agressivness (small more agressive)
    
        ## state variable to NRZI2NRZ
        self.NRZIprevBit = bool(1)  
        
        ## State variables for findPackets
        self.state = 'search'   # state variable: 'search' or 'pkt'
        self.pktcounter = 0   # counts the length of a packet
        self.packet = bitarray.bitarray([0,1,1,1,1,1,1,0]) # current packet being collected
        self.bitpointer = 0   # pointer to advance the search beyond what was already 
                              # searched in the previous buffer

        ## state variables for processBuffer
        self.buff = zeros(self.Nbuffer)   # large overlap-save buffer
        self.chunk_count = 0              # chunk counter
        # bits from end of prev buffer to be copied to beginning of new
        self.oldbits = bitarray.bitarray([0,0,0,0,0,0,0])    
        self.Npackets = 0                 # packet counter
        
    
    def NRZ2NRZI(self,NRZ, prevBit=True):
        NRZI = NRZ.copy() 
        for n in range(0,len(NRZ)):
            if NRZ[n] :
                NRZI[n] = prevBit
            else:
                NRZI[n] = not(prevBit)
            prevBit = NRZI[n]
        return NRZI
    

    def NRZI2NRZ(self, NRZI):  
        NRZ = NRZI.copy() 
        for n in range(0,len(NRZI)):
            NRZ[n] = NRZI[n] == self.NRZIprevBit
            self.NRZIprevBit = NRZI[n]
        return NRZ
    
    
    def KISS2bits(self,KISS):
        # function that takes a KISS frame sent via TCP/IP and converts it to an APRSpacket bit stream.
        bits = bitarray.bitarray(endian="little")
        bits.frombytes(KISS)
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
        
        bits.frombytes(fcs.digest())
        return bitarray.bitarray('01111110') + ax25.bit_stuff(bits) + bitarray.bitarray('01111110') 
     
        
    def bits2KISS(self,bits):
        # function that takes a bitstream of an APRS-packet, removes flags and FCS and unstuffs the bits
        bitsu = ax25.bit_unstuff(bits[8:-8])
        return  bitsu[:-16].tobytes() 
    
    
    def modulate(self,bits):
    # the function will take a bitarray of bits and will output an AFSK1200 modulated signal of them, 
    # sampled at fs Hz
    #  Inputs:
    #         bits  - bitarray of bits
    #         fs    - sampling rate
    # Outputs:
    #         sig    -  returns afsk1200 modulated signal
        # For you to complete
  
        
        return sig 
    
    
    def modulatePacket(self, callsign, digi, dest, info, preflags=80, postflags=80):
        # given callsign, digipath, dest, info, number of pre-flags and post-flags, the function contructs
        # an appropriate APRS packet, then converts it to NRZI and calls `modulate` 
        # to afsk1200 modulate the packet. 
        
        packet = ax25.UI(destination=dest,source=callsign, info=info, digipeaters=digi.split(b','),)
        prefix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(preflags,)).tolist())
        suffix = bitarray.bitarray(np.tile([0,1,1,1,1,1,1,0],(postflags,)).tolist())
        sig = self.modulate(self.NRZ2NRZI(prefix + packet.unparse()+suffix))

        return sig
    

    def demod(self, buff):
        # Demodulates a buffer and returns valid NRZ
    
        # Similar to afsk1200_demod, for you to complete

        
        return NRZ


    def FastPLL(self,NRZa):
        recbits = np.zeros(len(NRZa)//(self.fs//1200)*2,dtype=np.int32)
        pll = np.zeros(1,dtype = np.int32)
        pll[0] = self.pll
        ppll = np.zeros(1,dtype = np.int32)
        ppll[0] = self.ppll
        
        NRZb = (NRZa > 0).astype(np.int32)
        tot = libcd.pll(NRZb,len(NRZb),recbits,recbits,pll,ppll,self.dpll,self.apll)
        
        self.ppll = ppll.copy()
        self.pll = pll.copy()
        
        return bitarray.bitarray(recbits[:tot].tolist())
    
    
    def PLL(self, NRZa):
        idx = zeros(len(NRZa)//int(self.Ns)*2)   # allocate space to save indices        
        c = 0
        
        for n in range(1,len(NRZa)):
            if (self.pll < 0) and (self.ppll >0):
                idx[c] = n
                c = c+1
        
            if (NRZa[n] >= 0) !=  (NRZa[n-1] >=0):
                self.pll = np.int32(self.pll*self.apll)    
        
            self.ppll = self.pll
            self.pll = np.int32(self.pll+ self.dpll)
    
        return idx[:c].astype(np.int32) 
    

    def findPackets(self,bits):
        # function take a bitarray and looks for AX.25 packets in it. 
        # It implements a 2-state machine of searching for flag or collecting packets
        flg = bitarray.bitarray([0,1,1,1,1,1,1,0])
        packets = []
        n = self.bitpointer
        
        # Loop over bits
        while (n < len(bits)-7) :
            # default state is searching for packets
            if self.state is 'search':
                # look for 1111110, because can't be sure if the first zero is decoded
                # well if the packet is not padded.
                if bits[n:n+7] == flg[1:]:
                    # flag detected, so switch state to collecting bits in a packet
                    # start by copying the flag to the packet
                    # start counter to count the number of bits in the packet
                    self.state = 'pkt'
                    self.packet=flg.copy()
                    self.pktcounter = 8
                    # Advance to the end of the flag
                    n = n + 7
                else:
                    # flag was not found, advance by 1
                    n = n + 1             
            # state is to collect packet data. 
            elif self.state is 'pkt':
                # Check if we reached a flag by comparing with 0111111
                # 6 times ones is not allowed in a packet, hence it must be a flag (if there's no error)
                if bits[n:n+7] == flg[:7]:
                    # Flag detected, check if packet is longer than some minimum
                    if self.pktcounter > 200:
                        #print('packet found!')
                        # End of packet reached! append packet to list and switch to searching state
                        # We don't advance pointer since this our packet might have been
                        # flase detection and this flag could be the beginning of a real packet
                        self.state = 'search'
                        self.packet.extend(flg)
                        packets.append(self.packet.copy())
                    else:
                        # packet is too short! false alarm. Keep searching 
                        # We don't advance pointer since this this flag could be the beginning of a real packet
                        self.state = 'search'
                # No flag, so collect the bit and add to the packet
                else:
                    # check if packet is too long... if so, must be false alarm
                    if self.pktcounter < 2680:
                        # Not a false alarm, collect the bit and advance pointer        
                        self.packet.append(bits[n])
                        self.pktcounter = self.pktcounter + 1
                        n = n + 1
                    else:  #runaway packet
                        #runaway packet, switch state to searching, and advance pointer
                        self.state = 'search'
                        n = n + 1
        
        self.bitpointer = n-(len(bits)-7) 
        return packets

    
    # function to generate a checksum for validating packets
    def genfcs(self,bits):
        # Generates a checksum from packet bits
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
    
        digest = bitarray.bitarray(endian="little")
        digest.frombytes(fcs.digest())

        return digest


    # function to parse packet bits to information
    def decodeAX25(self,bits, deepsearch=False):
        ax = ax25.AX25()
        ax.info = "bad packet"
        ax.parse(bits)
        return ax
    
    
    def processBuffer(self, buff_in):
        # function processes an audio buffer. It collect several small into a large one
        # Then it demodulates and finds packets.
        #
        # The function uses overlap and save convolution
        # The function returns packets when they become available. Otherwise, returns empty list
        
        N = self.N
        NN = N*3 - 3
        
        Nchunks = self.Nchunks
        Abuffer = self.Abuffer
        fs = self.fs
        Ns = self.Ns
        
        validPackets=[]
        packets=[]
        NRZI=[]
        idx = []
        bits = []
        
        # Fill in buffer at the right place
        self.buff[NN+self.chunk_count*Abuffer:NN+(self.chunk_count+1)*Abuffer] = buff_in.copy()
        self.chunk_count = self.chunk_count + 1  
        
        # number of chunk reached -- process large buffer
        if self.chunk_count == Nchunks:
            # Demodulate to get NRZI
            NRZI = self.demod(self.buff)
            
            # compute sampling points, using PLL
            bits = self.FastPLL(NRZI)
            # In case that buffer is too small, raise an error -- must have at least 7 bits worth
            if len(bits) < 7:
                raise ValueError('number of bits too small for buffer')
            
            # concatenate end of previous buffer to current one
            bits = self.oldbits + self.NRZI2NRZ(bits)
            
            # store end of bit buffer to next buffer
            self.oldbits = bits[-7:].copy()
            
            # look for packets
            packets = self.findPackets(bits)
            
            # Copy end of sample buffer to the beginning of the next (overlap and save)
            self.buff[:NN] = self.buff[-NN:].copy()
            
            # reset chunk counter
            self.chunk_count = 0
            
            # checksum test for all detected packets
            for n in range(0,len(packets)):
                if len(packets[n]) > 200: 
                    try:
                        ax = self.decodeAX25(packets[n])
                    except:
                        ax = ax25.AX25()
                        ax.info = "bad packet"
                    if ax.info != 'bad packet' and ax.info != 'no decode':
                        validPackets.append(packets[n])                
  
        return validPackets

### Testing the modem reception

Now, we are ready to test our modem. Let's first load the ISS recording and see if our modem can detect the 24 packets we detected earlier. The difference is that we will load data in small buffers of 1024 samples and process them over a larger buffer made of several small buffers.  There are two points to make here. First: the idea is that no matter what size of buffers you use, the result of the overlap and save should be the same. If it isn't, then you have a bug.... and believe me... I had many before I got it right. Second: Generally, there are some overheads associated with operating on different sized buffers. The performances will vary with different architectures and memory.  We will look at several sizes and measure the time to process about 9 million samples (3.4 min at 44.1 kHz)  

#### Task 2:

* Load ISS.wav
* Create a TNC object using `modem = TNCaprs(fs=fs, Abuffer=1024, Nchunks=1)`
* Process the samples with the modem, 1024 samples at a time, using `packets = modem.processBuffer(sig[n:n+Abuffer])`
* The method `modem.processBuffer` will return a non-empty object whenever it detects packets. Iterate on the returned objects and display the packets.

Basically, you will call the demodulation and decoding code every 1024/44100 = 23 ms when `Nchunks=1`.

In [ ]:
fs, sig = wavread("ISS.wav")

In [ ]:
Abuffer = 1024
Nchunks = 1

modem = TNCaprs(fs=fs, Abuffer=Abuffer, Nchunks=Nchunks)

starttime = time.time()
npack = 0
for n in range(0,len(sig),Abuffer):
    packets = modem.processBuffer(sig[n:n+Abuffer])
    for pkt in packets: 
        npack = npack + 1
        ax = modem.decodeAX25(pkt)
        infostr = "%(n) 2i) | DEST: %(dest)s | SRC: %(src)s | DIGI: %(digi)s | %(info)s |" % {
                        'n': npack,
                        'dest': ax.destination,
                        'src': ax.source,
                        'digi': ax.digipeaters,
                        'info': ax.info.strip()
                    }
        print(infostr)
 
print("Processing took:", time.time()-starttime)

#### Task 3:
Repeat the task, but now use `Nchunks=10`. This means you will be performing demodulation and decoding on a buffer of size 1024*10 corresponding to 232 ms. Do you see improvement in the overall performance?

In [ ]:
# Repeat task above



#### Task 4: 
Now, repeat for `Nchunks=100` (\~2.3 s) -- the buffer size will also have associated delay before packets are presented, which in some situations can be a problem when timely response is needed. 

In [ ]:
# Repeat task above



### Sending Files in Loopback

We will define a specific format for sending files through APRS that will allow your course staff to receive and reconstruct them for display on a server. For each packet, the `source` will be your callsign + ssid. The `destination` will be APCAL (remember, the destination actually defines the version of software we are using). Packets will be of Message type with addressee yourself, your partner, or EE123.

The body of the first packet contains START, a 4-digit unique ID, a filename you want the received data to be saved as, and a comment (allowed to be blank) separated by commas. The info field thus looks like

``:TGTCALL :START####,filename,comment here``

`"{:04d}".format(uid)` ensures that your number is left-padded with 0s to fill 4 digits.

The bodies of subsequent packets are filled with data, followed by a 4-digit sequence number to allow reconstruction if packets arrive out of order at a receiver. Be careful not to exceed the length of the info field, as 15 bytes are taken up by the address and sequence number.

``:TGTCALL :<data data data>####``

The APRS standard requires that, except for certain standard protocols, data be printable. We therefore encode any binary data we send as [base64](https://en.wikipedia.org/wiki/Base64), a format which is printable and also satisfies the recommendation that messages be representable with 7-bit encodings. We have written a function ``file_to_b64`` for you below which reads in a file and converts it to base64. We have also provided ``ee123_bitarr_to_base64`` and ``ee123_base64_to_bitarr`` which convert between bitarrays (possibly not of integer byte length) and base64. They will be useful later in your project. 

_Note that each of these encoding functions prepends the number of subsequent bytes or bits as a check for reconstruction later. When you write your reconstruction function you must handle this (uint32) feature._

The final packet after all data has been sent contains END and the unique ID again.

``:YOURCALL :END####``

We will write code that does the following: reads a file, breaks it into packets, modulates them and plays them in loopback mode. At the same time, records the played packets, demodulates them, and stores the information frame containing the data into a new file. 
Basically, a full file transfer in loopback mode. 

We will use this protocol to send the below image.


In [ ]:
from PIL import Image
fname = "calBlue.tiff"
a = Image.open(fname)
plt.imshow(a)

#### Task 5:

* Connect the interface in loopback mode
* Read the `calBlue.tiff` file into a base64 bytes object with `file_to_b64`
* Create a modem using the TNCaprs with `fs=48000` Hz, `Abuffer=1024`, and `Nchunks=10`
* Read 256 bytes from the file at a time, create APRS packets with the 256 bytes in the information field
* Modulate the packets and push into a Queue named `Qout`

We have provided skeleton code for you in `enqueue_data`.


In [ ]:
# Convert from bitarray class to b64 bytes and back
def ee123_bitarr_to_base64(bits: bitarray.bitarray):
    bN = np.uint32(len(bits)).tobytes()
    Bytes = bN + bits.tobytes()
    return base64.b64encode(Bytes)

def ee123_base64_to_bitarr(b64: str):
    Bytes = base64.b64decode(b64)  # Be careful not to overwrite the builtin bytes class!
    N = np.frombuffer(Bytes[:4], dtype='<u4')[0]
    ba = bitarray.bitarray()
    ba.frombytes(Bytes[4:])
    return ba[:N]

def file_to_b64(fname):
    with open(fname, 'rb') as f:
        raw = f.read()
    # Prepend the number of bytes before encoding
    b64 = base64.b64encode(np.uint32(len(raw)).tobytes() + raw)
    return b64

def enqueue_data(callsign, modem, data, address=None, uid=None, fname="myfile.bin", 
                 comment="UCB EE123 message format", dest=b"APCAL", bsize=240):
    """
    Inputs:
      callsign: your callsign
      modem: a modem object
      data: data to send as a bytes object
      address: address callsign for the message, defaults to your callsign
      uid: optionally specify a UID, otherwise one will be randomly chosen
      fname: filename to send in start packet
      comment: comment for start packet
      dest: should be APCAL for EE123 data
      bsize: number of bytes to send per packet
      
    Outputs:
      Qout: a queue containing modulated packets to transmit
      uid: the uid used for this data transfer
    """
    if address is None:
        address = callsign
    if uid is None:
        uid = np.random.randint(0, 10000)
    uid = bytes("{:04d}".format(uid), 'utf-8')
    print("Putting packets in Queue for transmission ID=%s" % uid.decode())
    
    Qout = Queue.Queue()
    
    # For you to complete
    
    # Enqueue START packet
    
    # Enqueue data packets
    
    # Enqueue END packet
    
    return Qout, uid
    

In [ ]:
callsign = "??????-2"
fs = 48000
modem = TNCaprs(fs=fs, Abuffer=1024, Nchunks=10)
fname = "calBlue.tiff"

print("Putting packets in Queue")

# For you to complete: Enqueue the file



print("Done. UID=%s" % uid.decode())

print(f"There are {Qout.qsize()} packets in Qout")

#### Task 6:
We provide a replay callback similar to Lab 4. The callback stores samples from the loopback into a queue named ``Qin`` and also plays the samples on the specified audio device, so you can hear it. Don't forget to stop direwolf while running this section.

Start the record-play stream. While samples are being recorded and pushed to ``Qin`` in the background, read the packets from `Qout` and play each packet on the loopback audio using sd.play() with ``blocking=True``.

Warning: this will take approximately 1 minute and sound very annoying!

In [ ]:
# Define the callback factory so we can specify which non-global queue we want
def make_QR_callback(Qin):
    def queueREPLAY_callback(indata, outdata, frames, time, status):
#         assert frames == 1024
        if status:
            print(status)
        outdata[:] = indata
        Qin.put(indata.copy()[:, 0]) # Global queue
    return queueREPLAY_callback

In [ ]:
Qin = Queue.Queue()
# This stream captures samples on loopback_rcv and plays them on the built in device.
# Remember that loopback is connected to loopback_rcv.
st_replay = sd.Stream(device=(loop_idx_rcv, builtin_idx), blocksize=modem.Abuffer, callback=make_QR_callback(Qin))
print("Starting stream")
st_replay.start()
starttime = time.time()

# For you to complete

    
# Remember, we have to stop and close the stream.
st_replay.stop()
st_replay.close()

print(f"Done: {time.time() - starttime:.2f} s")
print(f"There are {Qin.qsize()} buffers in Qin")

In [ ]:
# Remember, we have to stop and close the stream.
st_replay.stop()
st_replay.close()

#### Task 7:
Now we will receive and reassemble a file transmitted over the loopback interface.

* Using the recording in Qin from Task 6, find and extract packets with ``modem.processBuffer``
* Decode the packets with ``modem.decodeAX25``
* Write a function ``decode_ee123_message`` to extract relevant information from the info field of a packet
* Record the unique ID of the transmission
* Collect the transmitted base64 data and decode it (you may want to use ``base64.b64decode``)
* Write the data to an appropriately named file
* Show the reconstructed image

In [ ]:
def decode_ee123_message(msg):
    """
    Input:
      msg: The info field of an APRS message packet
    
    Output:
      A dict containing the key components the packet may contain
      
    Message formats:
    :CALLSIGN :START####,filename,comment
    :CALLSIGN :<data data data>####
    :CALLSIGN :END####
    """
    msg = msg.strip()
    addressee = ''
    uid = None
    isstart = False
    isend = False
    seq = None
    data = None  # This should be a bytes object
    filename = ''
    comment = ''
    
    # Your code here
    
    
    
    return {'addr': addressee, 'uid': uid, 'isstart': isstart, 
            'isend': isend, 'seq': seq, 'filename': filename, 
            'data': data, 'comment': comment}

In [ ]:
# Decode the recording
print("decoding packets")
starttime = time.time()

# For you to complete

# Store the name of the reconstructed file in a variable called rec_fname




print(f"Done: {time.time() - starttime:.2f} s")


In [ ]:
from PIL import Image
path = f"rec_{rec_fname}"
a = Image.open(path)
plt.imshow(a)

#### Task 9:
Now repeat the above steps, but instead of sending the packet to yourself send it to your partner via direwolf.

* Manually copy your `direwolf-loopback.conf` from Part II into the Part III directory
* Start direwolf with `./run-direwolf-loopback.sh` in a terminal
* Generate a queue of messages to send to your partner, with their callsign as the message address
* Send the messages, with sufficient delays to avoid dropped messages (max 15/minute), over loopback to direwolf
* Your partner should be receiving and their direwolf position beacon should be within 50km of yours to pass the serverside filters
* You may want to adjust the direwolf config to set the local filter to ``t/m`` instead of ``t/mwnps`` so that your 15 packets per second are not taken by messages from other people
* Repeat these steps so that you receive a file from your partner

Part of the APRS protocol specifies that when a message received from APRS-IS is retransmitted over RF, it should be wrapped in a 3rd Party Header. See the [standard](http://www.aprs.org/doc/APRS101.PDF) page 95 for more details. The extra information in a 3rd Party Header takes up 20 or more bytes in the info field of an AX.25 packet, reducing the maximum number of bytes we can safely transmit in a single packet. However, the APRS-IS servers and, most importantly, direwolf are able to handle longer packets which break the standard for safety when dealing with unusual character encodings. We can take advantage of that fact for the purpose of this project to avoid truncating the data in each packet even more than we already have to. 

**DO NOT send messages of this length if you expect them to actually be transmitted over RF, as you could cause other hams' equipment to crash.** 

We know that our direwolf IGates are connected directly to the modem via a software loopback, so we are _probably_ ok. 

We have modified newax25.py to automatically recurse into 3rd Part Header packets and decode the nested packet, which contains the information we want for this project. If you write your own TNC code someday, be wary of this case.

In [ ]:
# Send a message to your partner

# Create the message queue
callsign = "??????-2"
partner_callsign = "??????"
fs = 48000
modem = TNCaprs(fs=fs, Abuffer=1024, Nchunks=10)
fname = ""

# For you to complete: Enqueue the file
print("Putting packets in Queue")




print("Done. UID=%s" % uid.decode())
    
# For you to complete: Modulate and send
print("Starting transmission")
starttime = time.time()




print(f"Done: {time.time() - starttime:.2f} s")


In [ ]:
# Receive a message from your partner

fs = 48000
modem = TNCaprs(fs=fs, Abuffer=1024, Nchunks=10)

Qin = Queue.Queue()
# This stream captures samples from direwolf on loopback and plays them on the built in device.
# Remember that loopback is connected to loopback_rcv.
st_replay = sd.Stream(device=(loop_idx, builtin_idx), blocksize=modem.Abuffer, callback=make_QR_callback(Qin))
print("Starting stream")
st_replay.start()
starttime = time.time()

print("decoding packets")

# For you to complete



        
print(f"Done: {time.time() - starttime:.2f} s")

In [ ]:
st_replay.stop()
st_replay.close()

#### Task 10:
Now we will disconnect direwolf from the public APRS network and instead direct it to our private server running in Cory Hall. The reason is to avoid overwhelming the public APRS network with data packets. We will do this by creating a new direwolf config file and running a different bash script that uses the new direwolf config file. The config file will be called `direwolf-server.conf` and the bash script is called `run-direwolf-server.sh`.

Now check your file transfer protocol for compatibility with our server by sending `text.png` to NU6XB, then checking our [server](http://igen2.eecs.berkeley.edu/packets) to make sure the file shows up and is able to be displayed in your browser. You may want to try sending a smaller file (like `hello.txt`) first for debugging.

In [ ]:
import os
cmd = "cat direwolf-loopback.conf  | sed  's/noam.aprs2.net/igen2.eecs.berkeley.edu/g' > direwolf-server.conf"
print(cmd)
os.system(cmd)

Stop any running instances of direwolf and run `./run-direwolf-server.sh` in a terminal. If you want to go back to previous tasks for debugging, don't forget to stop direwolf and do `./run-direwolf-loopback.sh`. In short, the "server" one connects to our private server, but the "loopback" one connects to the public APRS network.

In [ ]:
# Send to our server
print("Starting transmission")
starttime = time.time()

# For you to complete




print(f"Done: {time.time() - starttime:.2f} s")
